In [0]:
import dlt
from pyspark.sql.functions import col
from pyspark.sql.avro.functions import from_avro

In [0]:
# Parameters are typically passed via Pipeline settings
writer_schema = spark.conf.get('pipelines.writer_schema')

In [0]:
# constraints for data validataion
@dlt.expect("non_negative_amount", "amount >= 0")

In [0]:
# Table decleration with decorators
@dlt.table(comment="Decoded and cleansed (silver)")

In [0]:
def silver():
    b = dlt.read_stream('bronze')
    decoded = b.select(from_avro('value_bytes', writer_schema).alias('data'), 'kafka_ts','topic','partition','offset')
    flat = decoded.select(
        col('data.order_id').alias('order_id'),
        col('data.event_time').cast('timestamp').alias('event_ts'),
        col('data.customer_id').alias('customer_id'),
        col('data.amount').cast('double').alias('amount'),
        'kafka_ts','topic','partition','offset'
    )
    return flat